In [1]:
import os, sys
import glob
import re
import time

import geopandas as gpd 
import pandas as pd
from shapely.geometry import Point, LineString, Polygon, shape
from shapely.ops import voronoi_diagram
from shapely.validation import make_valid
from longsgis import voronoiDiagram4plg 
import fiona
import numpy

from xrspatial import zonal_stats 
import xarray as xr 
import rasterio 
import rioxarray 
from osgeo import gdal, osr, ogr

In [2]:
ProjectFolder = os.getcwd()
WorldPopFolder = os.path.join(ProjectFolder, "WorldPop_tifs_albers")
print(ProjectFolder)
print(WorldPopFolder)

C:\Users\wb527163\GEO_Cdrive_Grace\urban_growth
C:\Users\wb527163\GEO_Cdrive_Grace\urban_growth\WorldPop_tifs_albers


In [23]:
def zonal_stats(feat, input_zone_polygon, input_value_raster, statistic='sum'):

    # Open data
    raster = gdal.Open(input_value_raster)
    shp = ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()

    # Get raster georeference info
    transform = raster.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]

    # Get extent of feat
    geom = feat.GetGeometryRef()
    if (geom.GetGeometryName() == 'MULTIPOLYGON'):
        count = 0
        pointsX = []; pointsY = []
        for polygon in geom:
            geomInner = geom.GetGeometryRef(count)    
            ring = geomInner.GetGeometryRef(0)
            numpoints = ring.GetPointCount()
            for p in range(numpoints):
                    lon, lat, z = ring.GetPoint(p)
                    pointsX.append(lon)
                    pointsY.append(lat)    
            count += 1
    elif (geom.GetGeometryName() == 'POLYGON'):
        ring = geom.GetGeometryRef(0)
        numpoints = ring.GetPointCount()
        pointsX = []; pointsY = []
        for p in range(numpoints):
                lon, lat, z = ring.GetPoint(p)
                pointsX.append(lon)
                pointsY.append(lat)

    else:
        sys.exit()

    xmin = min(pointsX)
    xmax = max(pointsX)
    ymin = min(pointsY)
    ymax = max(pointsY)

    # Specify offset and rows and columns to read
    xoff = int((xmin - xOrigin)/pixelWidth)
    yoff = int((yOrigin - ymax)/pixelWidth)
    xcount = int((xmax - xmin)/pixelWidth)+1
    ycount = int((ymax - ymin)/pixelWidth)+1

    # Create memory target raster
    target_ds = gdal.GetDriverByName('MEM').Create('', xcount, ycount, gdal.GDT_Byte)
    target_ds.SetGeoTransform((
        xmin, pixelWidth, 0,
        ymax, 0, pixelHeight,
    ))

    # Create for target raster the same projection as for the value raster
    raster_srs = osr.SpatialReference()
    raster_srs.ImportFromWkt(raster.GetProjectionRef())
    target_ds.SetProjection(raster_srs.ExportToWkt())

    # Rasterize zone polygon to raster
    gdal.RasterizeLayer(target_ds, [1], lyr, burn_values=[1])

    # Read raster as arrays
    banddataraster = raster.GetRasterBand(1)
    dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(numpy.float)

    bandmask = target_ds.GetRasterBand(1)
    datamask = bandmask.ReadAsArray(0, 0, xcount, ycount).astype(numpy.float)

    # Mask zone of raster
    zoneraster = numpy.ma.masked_array(dataraster,  numpy.logical_not(datamask))

    # Calculate statistics of zonal raster
    if (statistic == 'sum'):
        return numpy.sum(zoneraster)
    elif (statistic == 'mean'):
        return numpy.mean(zoneraster)
    elif (statistic == 'count'):
        return numpy.ma.count(zoneraster)
    elif (statistic == 'median'):
        return numpy.median(zoneraster)
    elif (statistic == 'min'):
        return numpy.min(zoneraster)
    elif (statistic == 'max'):
        return numpy.max(zoneraster)
    elif (statistic == 'std'):
        return numpy.std(zoneraster)
    else:
        return numpy.sum(zoneraster)

In [24]:
def loop_zonal_stats(input_zone_polygon, input_value_raster, statistic):

    shp = ogr.Open(input_zone_polygon)
    lyr = shp.GetLayer()
    featList = range(lyr.GetFeatureCount())
    statDict = {}

    for FID in featList:
        feat = lyr.GetFeature(FID)
        sumValue = zonal_stats(feat, input_zone_polygon, input_value_raster, statistic)
        statDict[FID] = sumValue
    return statDict

In [28]:
# Raster dataset
input_value_raster = r'test2003_ras.tif'
# Vector dataset(zones)
input_zone_polygon = 'test2003_ply.shp'

Year2003 = loop_zonal_stats(input_zone_polygon, input_value_raster, statistic='min')
Year2003

C:\Users\wb527163\AppData\Local\Temp\ipykernel_14268\4246996182.py:69: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataraster = banddataraster.ReadAsArray(xoff, yoff, xcount, ycount).astype(numpy.float)
C:\Users\wb527163\AppData\Local\Temp\ipykernel_14268\4246996182.py:72: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  datamask = bandmas

{0: 0.7867384552955627,
 1: 0.640760600566864,
 2: 0.1967393010854721,
 3: 0.38563650846481323,
 4: 0.32887518405914307,
 5: 0.22637289762496948,
 6: 0.22637289762496948,
 7: 0.8812860250473022,
 8: 0.49483421444892883,
 9: 0.23517312109470367,
 10: 0.5259771943092346,
 11: 1.8976683616638184,
 12: 0.5722252130508423,
 13: 0.40990403294563293,
 14: 0.26727160811424255,
 15: 0.3469986915588379,
 16: 0.25471776723861694,
 17: 0.7260482311248779,
 18: 0.33341720700263977,
 19: 0.3848814070224762,
 20: 0.48839154839515686,
 21: 0.38408806920051575,
 22: 0.33890819549560547,
 23: 0.34060901403427124,
 24: 0.20294594764709473,
 25: 0.39755725860595703,
 26: 0.2872084081172943,
 27: 0.34090113639831543,
 28: 0.2110586166381836,
 29: 0.42498138546943665,
 30: 0.4029380977153778,
 31: 0.21892909705638885,
 32: 0.2815491855144501,
 33: 0.21832071244716644,
 34: 0.4224005937576294,
 35: 0.47695979475975037,
 36: 0.20825420320034027,
 37: 0.20858672261238098,
 38: 0.20825420320034027,
 39: 0.40503

In [13]:
poly = gpd.read_file('test2003_ply.shp')
poly

,FID_1,index_righ,GRID3_spli,MAX_year,Shape_Leng,Shape_Area,geometry
0,3525,4043,27634.0,1985,282.4717,4787.414548,"POLYGON ((-1386111.556 504906.199, -1386224.54..."
1,3526,4047,27722.0,2002,903.9091,7181.117585,"POLYGON ((-1386798.600 506714.017, -1386851.44..."
2,3527,4048,27741.0,2002,677.9319,20745.452685,"POLYGON ((-1375504.749 508272.320, -1375237.48..."
3,8079,4027,27508.0,1985,112.9886,797.901483,"POLYGON ((-1412621.514 501926.124, -1412864.18..."
4,8089,4036,27578.0,1985,677.9321,14362.249294,"POLYGON ((-1413969.204 502763.217, -1414023.34..."
...,...,...,...,...,...,...,...
242,8963,3952,26375.0,1985,790.9202,24734.965749,"POLYGON ((-1415417.982 472639.117, -1415497.42..."
243,8964,3936,26189.0,1985,169.4827,1595.798729,"POLYGON ((-1410744.562 471863.136, -1411562.24..."
244,8968,3947,26354.0,2001,1242.8749,39895.100991,"POLYGON ((-1412409.660 473671.588, -1412787.77..."
245,8975,8803,190976.0,2003,790.9205,19947.549790,"POLYGON ((-1415488.600 474681.740, -1415718.22..."


In [15]:
zstats = pd.DataFrame.from_dict(Year2003, orient = 'index', columns = ['Pop2003'])
poly = poly.merge(zstats, how='left', left_index=True, right_index=True)
poly

,FID_1,index_righ,GRID3_spli,MAX_year,Shape_Leng,Shape_Area,geometry,Pop2003
0,3525,4043,27634.0,1985,282.4717,4787.414548,"POLYGON ((-1386111.556 504906.199, -1386224.54...",897.964939
1,3526,4047,27722.0,2002,903.9091,7181.117585,"POLYGON ((-1386798.600 506714.017, -1386851.44...",1685.139659
2,3527,4048,27741.0,2002,677.9319,20745.452685,"POLYGON ((-1375504.749 508272.320, -1375237.48...",475.820697
3,8079,4027,27508.0,1985,112.9886,797.901483,"POLYGON ((-1412621.514 501926.124, -1412864.18...",145.908172
4,8089,4036,27578.0,1985,677.9321,14362.249294,"POLYGON ((-1413969.204 502763.217, -1414023.34...",168.651912
...,...,...,...,...,...,...,...,...
242,8963,3952,26375.0,1985,790.9202,24734.965749,"POLYGON ((-1415417.982 472639.117, -1415497.42...",874.326976
243,8964,3936,26189.0,1985,169.4827,1595.798729,"POLYGON ((-1410744.562 471863.136, -1411562.24...",258.248996
244,8968,3947,26354.0,2001,1242.8749,39895.100991,"POLYGON ((-1412409.660 473671.588, -1412787.77...",553.813762
245,8975,8803,190976.0,2003,790.9205,19947.549790,"POLYGON ((-1415488.600 474681.740, -1415718.22...",230.075384
